
# **８．フレーム構造解析コード**



## **8.1　単純梁の解析**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 1. 可視化関数 (Y軸自動ズーム・実数値表示版)
# ==========================================
def plot_frame_forces(nodes, elements, U_all, element_forces, dist_loads={}, scale_factor=1.0):
    """
    解析結果（変形図、SFD、BMD）を描画する関数
    ※Y軸の表示範囲を自動調整し、倍率1.0でも変形形状を確認できるようにしています。
    """
    plt.figure(figsize=(10, 12))

    # --- (1) 変形図 ---
    plt.subplot(3, 1, 1)

    nodes_def = {}
    y_coords_def = [] # Y軸範囲決定用のリスト

    # 変形後座標の計算
    for i, (x, y) in nodes.items():
        u = U_all[(i-1)*3]; v = U_all[(i-1)*3 + 1]

        # 実寸(scale_factor=1.0)で座標計算
        dx_val = x + u * scale_factor
        dy_val = y + v * scale_factor

        nodes_def[i] = (dx_val, dy_val)
        y_coords_def.append(dy_val)

    # 描画ループ
    for el_id, (i, j) in elements.items():
        # 変形前（点線）
        x = [nodes[i][0], nodes[j][0]]; y = [nodes[i][1], nodes[j][1]]
        plt.plot(x, y, 'k--', lw=1, alpha=0.3)
        # 変形後（赤実線）
        x_def = [nodes_def[i][0], nodes_def[j][0]]; y_def = [nodes_def[i][1], nodes_def[j][1]]
        plt.plot(x_def, y_def, 'r-', lw=2)

    plt.title(f'Deformation (Scale: {scale_factor}x / Real Value)')
    plt.grid(True)
    plt.ylabel('Y (mm)')

    # ★重要：Y軸範囲の自動調整ロジック（微小変形を見やすくする）
    y_min, y_max = min(y_coords_def), max(y_coords_def)
    y_range = y_max - y_min
    if y_range == 0: y_range = 1.0 # 変形ゼロの場合のゼロ割防止

    # 上下に少し余白を持たせてズーム
    margin = y_range * 0.2
    plt.ylim(y_min - margin, y_max + margin)

    # ※アスペクト比固定(equal)は解除しています

    # --- 内部関数: 部材内の分布(Q, M)を計算 ---
    def get_element_distribution(el_id, i, j, forces, w):
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)

        Q_i = forces['Q_i']; M_i = forces['M_i']

        num_points = 51
        x_local = np.linspace(0, L, num_points)

        # 断面力 Q(x), M(x)
        Q_vals = Q_i + w * x_local
        M_vals = -M_i + Q_i * x_local + 0.5 * w * x_local**2

        X_vals = np.linspace(xi, xj, num_points)
        return X_vals, Q_vals, M_vals

    # --- (2) SFD (せん断力図) ---
    plt.subplot(3, 1, 2)
    for el_id, (i, j) in elements.items():
        w = dist_loads.get(el_id, 0.0)
        X_vals, Q_vals, _ = get_element_distribution(el_id, i, j, element_forces[el_id], w)

        plt.plot(X_vals, [0]*len(X_vals), 'k-', lw=0.5, alpha=0.5)
        plt.plot(X_vals, Q_vals, 'b-', lw=2)
        plt.fill_between(X_vals, 0, Q_vals, alpha=0.2, color='blue')

        mid = len(X_vals)//2
        plt.text(X_vals[mid], Q_vals[mid], f'{Q_vals[mid]:.1f}', color='blue', fontsize=9)

    plt.title('Shear Force Diagram (SFD) [N]')
    plt.grid(True); plt.ylabel('Shear (N)')

    # --- (3) BMD (曲げモーメント図) ---
    plt.subplot(3, 1, 3)
    for el_id, (i, j) in elements.items():
        w = dist_loads.get(el_id, 0.0)
        X_vals, _, M_vals = get_element_distribution(el_id, i, j, element_forces[el_id], w)

        # 引張側を正（下側）に描画するため符号反転
        plt.plot(X_vals, [0]*len(X_vals), 'k-', lw=0.5, alpha=0.5)
        plt.plot(X_vals, -M_vals, 'g-', lw=2)
        plt.fill_between(X_vals, 0, -M_vals, alpha=0.2, color='green')

        mid = len(X_vals)//2
        val_mid = -M_vals[mid]
        plt.text(X_vals[mid], val_mid, f'{val_mid:.1f}', color='green', fontsize=9, ha='center')

    plt.title('Bending Moment Diagram (BMD) [N.mm] (Tension Side)')
    plt.grid(True); plt.xlabel('X (mm)'); plt.ylabel('Moment')

    plt.tight_layout()
    plt.show()

# ==========================================
# 2. フレーム解析ソルバー
# ==========================================
def solve_frame(nodes, elements, materials, bcs, loads, dist_loads={}):
    print("--- 解析開始 ---")
    num_nodes = len(nodes); num_dofs = num_nodes * 3
    K_all = np.zeros((num_dofs, num_dofs)); F_all = np.zeros(num_dofs)

    # 1. 剛性行列アセンブリ
    for el_id, (i, j) in elements.items():
        E = materials[el_id]['E']; A = materials[el_id]['A']; I = materials[el_id]['I']
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)
        c = dx / L; s = dy / L

        k_local = np.zeros((6, 6))
        k_local[0,0] = k_local[3,3] = E*A/L; k_local[0,3] = k_local[3,0] = -E*A/L
        k1 = 12*E*I/L**3; k2 = 6*E*I/L**2; k3 = 4*E*I/L; k4 = 2*E*I/L
        k_local[1,1]=k_local[4,4]=k1; k_local[1,4]=k_local[4,1]=-k1
        k_local[1,2]=k_local[2,1]=k2; k_local[1,5]=k_local[5,1]=k2
        k_local[4,2]=k_local[2,4]=-k2; k_local[4,5]=k_local[5,4]=-k2
        k_local[2,2]=k_local[5,5]=k3; k_local[2,5]=k_local[5,2]=k4

        T = np.zeros((6, 6)); rot = np.array([[c, s, 0], [-s, c, 0], [0, 0, 1]])
        T[0:3, 0:3] = rot; T[3:6, 3:6] = rot
        K_el = T.T @ k_local @ T

        indices = [ (i-1)*3, (i-1)*3+1, (i-1)*3+2, (j-1)*3, (j-1)*3+1, (j-1)*3+2 ]
        for r in range(6):
            for c_ in range(6):
                K_all[indices[r], indices[c_]] += K_el[r, c_]

    # 2. 荷重ベクトル作成
    # (a) 集中荷重
    for node_id, (fx, fy, mz) in loads.items():
        idx = (node_id - 1) * 3
        F_all[idx] += fx; F_all[idx+1] += fy; F_all[idx+2] += mz

    # (b) 分布荷重 -> 等価節点荷重
    for el_id, w in dist_loads.items():
        if w == 0: continue
        i, j = elements[el_id]
        xi, yi = nodes[i]; xj, yj = nodes[j]
        L = np.sqrt((xj-xi)**2 + (yj-yi)**2)

        Fy_eq = w * L / 2.0
        Mi_eq = -w * L**2 / 12.0
        Mj_eq = +w * L**2 / 12.0

        idx_i = (i-1)*3; idx_j = (j-1)*3
        F_all[idx_i+1] += Fy_eq; F_all[idx_i+2] += Mi_eq
        F_all[idx_j+1] += Fy_eq; F_all[idx_j+2] += Mj_eq

    # 3. 求解
    fixed_dofs = []
    for node_id, constraint in bcs.items():
        idx = (node_id - 1) * 3
        if constraint[0]: fixed_dofs.append(idx)
        if constraint[1]: fixed_dofs.append(idx+1)
        if constraint[2]: fixed_dofs.append(idx+2)

    free_dofs = [i for i in range(num_dofs) if i not in fixed_dofs]
    K_ff = K_all[np.ix_(free_dofs, free_dofs)]
    F_f = F_all[free_dofs]

    try:
        U_f = np.linalg.solve(K_ff, F_f)
    except:
        print("Error: Singular Matrix"); return None, None

    U_all = np.zeros(num_dofs); U_all[free_dofs] = U_f

    # 4. 断面力計算
    element_forces = {}
    for el_id, (i, j) in elements.items():
        E = materials[el_id]['E']; A = materials[el_id]['A']; I = materials[el_id]['I']
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)
        c = dx / L; s = dy / L

        u_vec = np.array([U_all[(i-1)*3], U_all[(i-1)*3+1], U_all[(i-1)*3+2],
                          U_all[(j-1)*3], U_all[(j-1)*3+1], U_all[(j-1)*3+2]])
        T = np.zeros((6, 6)); rot = np.array([[c, s, 0], [-s, c, 0], [0, 0, 1]])
        T[0:3, 0:3] = rot; T[3:6, 3:6] = rot
        u_local = T @ u_vec

        k_local = np.zeros((6, 6))
        k_local[0,0] = k_local[3,3] = E*A/L; k_local[0,3] = k_local[3,0] = -E*A/L
        k1 = 12*E*I/L**3; k2 = 6*E*I/L**2; k3 = 4*E*I/L; k4 = 2*E*I/L
        k_local[1,1]=k_local[4,4]=k1; k_local[1,4]=k_local[4,1]=-k1
        k_local[1,2]=k_local[2,1]=k2; k_local[1,5]=k_local[5,1]=k2
        k_local[4,2]=k_local[2,4]=-k2; k_local[4,5]=k_local[5,4]=-k2
        k_local[2,2]=k_local[5,5]=k3; k_local[2,5]=k_local[5,2]=k4

        f_disp = k_local @ u_local

        # 固定端反力の補正
        w = dist_loads.get(el_id, 0.0)
        f_fixed = np.zeros(6)
        if w != 0:
            Ry = -w * L / 2.0
            Mi = +w * L**2 / 12.0
            Mj = -w * L**2 / 12.0
            f_fixed = np.array([0, Ry, Mi, 0, Ry, Mj])

        f_total = f_disp + f_fixed

        element_forces[el_id] = {
            'N_j': f_total[3],
            'Q_i': f_total[1], 'M_i': f_total[2],
            'Q_j': f_total[4], 'M_j': f_total[5]
        }

    return U_all, element_forces

# ==========================================
# 3. メイン実行ブロック
# ==========================================
if __name__ == "__main__":
    # --- データ定義 (単位: N, mm) ---
    nodes = { 1: (0.0, 0.0), 2: (1000.0, 0.0), 3: (2000.0, 0.0) }
    elements = { 1: (1, 2), 2: (2, 3) }

    mat_props = { 'E': 2.0e5, 'A': 6.0e4, 'I': 4.5e8 }
    materials = { 1: mat_props, 2: mat_props }

    bcs = { 1: (True, True, False), 2: (False, False, False), 3: (False, True, False) }

    # 荷重 (中央点2に P=-10000 N)
    loads = { 1: (0.0, 0.0, 0.0), 2: (0.0, -10000.0, 0.0), 3: (0.0, 0.0, 0.0) }
    dist_loads = { 1: 0.0, 2: 0.0 }

    # 解析実行
    U, F = solve_frame(nodes, elements, materials, bcs, loads, dist_loads)

    if U is not None:
        # ★倍率は 1.0 (実寸) に設定
        scale_factor = 1.0

        print(f"Visualization Scale Factor: {scale_factor}x (Real Scale)")

        # 描画実行
        plot_frame_forces(nodes, elements, U, F, dist_loads, scale_factor=scale_factor)

        # --- 理論値との検証 ---
        P_val = 10000.0; L_val = 2000.0; E_val = 2.0e5; I_val = 4.5e8
        delta_theory = (P_val * L_val**3) / (48 * E_val * I_val)

        print(f"\n--- 検証 ---")
        print(f"中央たわみ (解析値): {abs(U[4]):.5f} mm")
        print(f"中央たわみ (理論値): {delta_theory:.5f} mm")

## **8.2 節点数を増やしてみる**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 1. 可視化関数 (Y軸自動ズーム・実数値表示版)
# ==========================================
def plot_frame_forces(nodes, elements, U_all, element_forces, dist_loads={}, scale_factor=1.0):
    """
    解析結果（変形図、SFD、BMD）を描画する関数
    ※Y軸の表示範囲を自動調整し、倍率1.0でも変形形状を確認できるようにしています。
    """
    plt.figure(figsize=(10, 12))

    # --- (1) 変形図 ---
    plt.subplot(3, 1, 1)

    nodes_def = {}
    y_coords_def = [] # Y軸範囲決定用のリスト

    # 変形後座標の計算
    for i, (x, y) in nodes.items():
        u = U_all[(i-1)*3]; v = U_all[(i-1)*3 + 1]

        # 実寸(scale_factor=1.0)で座標計算
        dx_val = x + u * scale_factor
        dy_val = y + v * scale_factor

        nodes_def[i] = (dx_val, dy_val)
        y_coords_def.append(dy_val)

    # 描画ループ
    for el_id, (i, j) in elements.items():
        # 変形前（点線）
        x = [nodes[i][0], nodes[j][0]]; y = [nodes[i][1], nodes[j][1]]
        plt.plot(x, y, 'k--', lw=1, alpha=0.3)
        # 変形後（赤実線）
        x_def = [nodes_def[i][0], nodes_def[j][0]]; y_def = [nodes_def[i][1], nodes_def[j][1]]
        plt.plot(x_def, y_def, 'r-', lw=2)

    plt.title(f'Deformation (Scale: {scale_factor}x / Real Value)')
    plt.grid(True)
    plt.ylabel('Y (mm)')

    # ★重要：Y軸範囲の自動調整ロジック（微小変形を見やすくする）
    y_min, y_max = min(y_coords_def), max(y_coords_def)
    y_range = y_max - y_min
    if y_range == 0: y_range = 1.0 # 変形ゼロの場合のゼロ割防止

    # 上下に少し余白を持たせてズーム
    margin = y_range * 0.2
    plt.ylim(y_min - margin, y_max + margin)

    # ※アスペクト比固定(equal)は解除しています

    # --- 内部関数: 部材内の分布(Q, M)を計算 ---
    def get_element_distribution(el_id, i, j, forces, w):
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)

        Q_i = forces['Q_i']; M_i = forces['M_i']

        num_points = 51
        x_local = np.linspace(0, L, num_points)

        # 断面力 Q(x), M(x)
        Q_vals = Q_i + w * x_local
        M_vals = -M_i + Q_i * x_local + 0.5 * w * x_local**2

        X_vals = np.linspace(xi, xj, num_points)
        return X_vals, Q_vals, M_vals

    # --- (2) SFD (せん断力図) ---
    plt.subplot(3, 1, 2)
    for el_id, (i, j) in elements.items():
        w = dist_loads.get(el_id, 0.0)
        X_vals, Q_vals, _ = get_element_distribution(el_id, i, j, element_forces[el_id], w)

        plt.plot(X_vals, [0]*len(X_vals), 'k-', lw=0.5, alpha=0.5)
        plt.plot(X_vals, Q_vals, 'b-', lw=2)
        plt.fill_between(X_vals, 0, Q_vals, alpha=0.2, color='blue')

        mid = len(X_vals)//2
        plt.text(X_vals[mid], Q_vals[mid], f'{Q_vals[mid]:.1f}', color='blue', fontsize=9)

    plt.title('Shear Force Diagram (SFD) [N]')
    plt.grid(True); plt.ylabel('Shear (N)')

    # --- (3) BMD (曲げモーメント図) ---
    plt.subplot(3, 1, 3)
    for el_id, (i, j) in elements.items():
        w = dist_loads.get(el_id, 0.0)
        X_vals, _, M_vals = get_element_distribution(el_id, i, j, element_forces[el_id], w)

        # 引張側を正（下側）に描画するため符号反転
        plt.plot(X_vals, [0]*len(X_vals), 'k-', lw=0.5, alpha=0.5)
        plt.plot(X_vals, -M_vals, 'g-', lw=2)
        plt.fill_between(X_vals, 0, -M_vals, alpha=0.2, color='green')

        mid = len(X_vals)//2
        val_mid = -M_vals[mid]
        plt.text(X_vals[mid], val_mid, f'{val_mid:.1f}', color='green', fontsize=9, ha='center')

    plt.title('Bending Moment Diagram (BMD) [N.mm] (Tension Side)')
    plt.grid(True); plt.xlabel('X (mm)'); plt.ylabel('Moment')

    plt.tight_layout()
    plt.show()

# ==========================================
# 2. フレーム解析ソルバー
# ==========================================
def solve_frame(nodes, elements, materials, bcs, loads, dist_loads={}):
    print("--- 解析開始 ---")
    num_nodes = len(nodes); num_dofs = num_nodes * 3
    K_all = np.zeros((num_dofs, num_dofs)); F_all = np.zeros(num_dofs)

    # 1. 剛性行列アセンブリ
    for el_id, (i, j) in elements.items():
        E = materials[el_id]['E']; A = materials[el_id]['A']; I = materials[el_id]['I']
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)
        c = dx / L; s = dy / L

        k_local = np.zeros((6, 6))
        k_local[0,0] = k_local[3,3] = E*A/L; k_local[0,3] = k_local[3,0] = -E*A/L
        k1 = 12*E*I/L**3; k2 = 6*E*I/L**2; k3 = 4*E*I/L; k4 = 2*E*I/L
        k_local[1,1]=k_local[4,4]=k1; k_local[1,4]=k_local[4,1]=-k1
        k_local[1,2]=k_local[2,1]=k2; k_local[1,5]=k_local[5,1]=k2
        k_local[4,2]=k_local[2,4]=-k2; k_local[4,5]=k_local[5,4]=-k2
        k_local[2,2]=k_local[5,5]=k3; k_local[2,5]=k_local[5,2]=k4

        T = np.zeros((6, 6)); rot = np.array([[c, s, 0], [-s, c, 0], [0, 0, 1]])
        T[0:3, 0:3] = rot; T[3:6, 3:6] = rot
        K_el = T.T @ k_local @ T

        indices = [ (i-1)*3, (i-1)*3+1, (i-1)*3+2, (j-1)*3, (j-1)*3+1, (j-1)*3+2 ]
        for r in range(6):
            for c_ in range(6):
                K_all[indices[r], indices[c_]] += K_el[r, c_]

    # 2. 荷重ベクトル作成
    # (a) 集中荷重
    for node_id, (fx, fy, mz) in loads.items():
        idx = (node_id - 1) * 3
        F_all[idx] += fx; F_all[idx+1] += fy; F_all[idx+2] += mz

    # (b) 分布荷重 -> 等価節点荷重
    for el_id, w in dist_loads.items():
        if w == 0: continue
        i, j = elements[el_id]
        xi, yi = nodes[i]; xj, yj = nodes[j]
        L = np.sqrt((xj-xi)**2 + (yj-yi)**2)

        Fy_eq = w * L / 2.0
        Mi_eq = -w * L**2 / 12.0
        Mj_eq = +w * L**2 / 12.0

        idx_i = (i-1)*3; idx_j = (j-1)*3
        F_all[idx_i+1] += Fy_eq; F_all[idx_i+2] += Mi_eq
        F_all[idx_j+1] += Fy_eq; F_all[idx_j+2] += Mj_eq

    # 3. 求解
    fixed_dofs = []
    for node_id, constraint in bcs.items():
        idx = (node_id - 1) * 3
        if constraint[0]: fixed_dofs.append(idx)
        if constraint[1]: fixed_dofs.append(idx+1)
        if constraint[2]: fixed_dofs.append(idx+2)

    free_dofs = [i for i in range(num_dofs) if i not in fixed_dofs]
    K_ff = K_all[np.ix_(free_dofs, free_dofs)]
    F_f = F_all[free_dofs]

    try:
        U_f = np.linalg.solve(K_ff, F_f)
    except:
        print("Error: Singular Matrix"); return None, None

    U_all = np.zeros(num_dofs); U_all[free_dofs] = U_f

    # 4. 断面力計算
    element_forces = {}
    for el_id, (i, j) in elements.items():
        E = materials[el_id]['E']; A = materials[el_id]['A']; I = materials[el_id]['I']
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)
        c = dx / L; s = dy / L

        u_vec = np.array([U_all[(i-1)*3], U_all[(i-1)*3+1], U_all[(i-1)*3+2],
                          U_all[(j-1)*3], U_all[(j-1)*3+1], U_all[(j-1)*3+2]])
        T = np.zeros((6, 6)); rot = np.array([[c, s, 0], [-s, c, 0], [0, 0, 1]])
        T[0:3, 0:3] = rot; T[3:6, 3:6] = rot
        u_local = T @ u_vec

        k_local = np.zeros((6, 6))
        k_local[0,0] = k_local[3,3] = E*A/L; k_local[0,3] = k_local[3,0] = -E*A/L
        k1 = 12*E*I/L**3; k2 = 6*E*I/L**2; k3 = 4*E*I/L; k4 = 2*E*I/L
        k_local[1,1]=k_local[4,4]=k1; k_local[1,4]=k_local[4,1]=-k1
        k_local[1,2]=k_local[2,1]=k2; k_local[1,5]=k_local[5,1]=k2
        k_local[4,2]=k_local[2,4]=-k2; k_local[4,5]=k_local[5,4]=-k2
        k_local[2,2]=k_local[5,5]=k3; k_local[2,5]=k_local[5,2]=k4

        f_disp = k_local @ u_local

        # 固定端反力の補正
        w = dist_loads.get(el_id, 0.0)
        f_fixed = np.zeros(6)
        if w != 0:
            Ry = -w * L / 2.0
            Mi = +w * L**2 / 12.0
            Mj = -w * L**2 / 12.0
            f_fixed = np.array([0, Ry, Mi, 0, Ry, Mj])

        f_total = f_disp + f_fixed

        element_forces[el_id] = {
            'N_j': f_total[3],
            'Q_i': f_total[1], 'M_i': f_total[2],
            'Q_j': f_total[4], 'M_j': f_total[5]
        }

    return U_all, element_forces

# ==========================================
# 3. メイン実行ブロック (メッシュ生成対応版)
# ==========================================
if __name__ == "__main__":
    import math

    # --- 解析パラメータ設定 ---
    L_total = 2000.0  # 全長 (mm)
    P_load = -10000.0 # 中央集中荷重 (N)

    # ★ここを変えて遊んでみよう！★
    num_div = 10      # 要素分割数（※中央に節点が来るよう「偶数」推奨）

    # --- 1. 節点・要素・材料の自動生成 ---
    nodes = {}
    elements = {}
    materials = {}
    dist_loads = {} # 分布荷重用（今回は0）

    # 材料定数
    mat_props = { 'E': 2.0e5, 'A': 6.0e4, 'I': 4.5e8 }

    dx = L_total / num_div  # 1要素あたりの長さ

    for i in range(num_div + 1):
        # 節点生成: (x, y)
        nodes[i+1] = (dx * i, 0.0)

        # 要素生成: 最後の節点以外で要素を作る
        if i < num_div:
            el_id = i + 1
            elements[el_id] = (i+1, i+2)
            materials[el_id] = mat_props
            dist_loads[el_id] = 0.0

    # --- 2. 境界条件 (左端:ピン, 右端:ローラー) ---
    bcs = {}
    # 左端 (Node 1): 固定 (x, y固定, 回転自由)
    bcs[1] = (True, True, False)
    # 右端 (Node N+1): ローラー (y固定)
    bcs[num_div + 1] = (False, True, False)
    # それ以外の中間節点: フリー (拘束なし)
    for i in range(2, num_div + 1):
        bcs[i] = (False, False, False)

    # --- 3. 荷重条件 (中央点に集中荷重) ---
    loads = {}
    # 全ての節点で荷重を初期化(0)
    for i in range(1, num_div + 2):
        loads[i] = (0.0, 0.0, 0.0)

    # 中央の節点を探して荷重を載せる
    # L/2 の位置にある節点を探す（誤差を考慮して判定）
    center_x = L_total / 2.0
    target_node = -1
    for nid, (nx, ny) in nodes.items():
        if abs(nx - center_x) < 1.0e-5: # ほぼ一致したら
            target_node = nid
            break

    if target_node != -1:
        loads[target_node] = (0.0, P_load, 0.0)
        print(f"Node {target_node} (x={nodes[target_node][0]}) に荷重 {P_load} N を載荷しました。")
    else:
        print("警告: 中央に節点がありません。分割数を「偶数」にしてください。")

    # --- 4. 解析実行 ---
    U, F = solve_frame(nodes, elements, materials, bcs, loads, dist_loads)

    if U is not None:
        # 倍率は1.0 (実寸)
        scale_factor = 1.0
        print(f"Visualization Scale Factor: {scale_factor}x (Real Scale)")

        # 描画 (節点が増えたので、繋ぐだけで滑らかに見えるはずです)
        plot_frame_forces(nodes, elements, U, F, dist_loads, scale_factor=scale_factor)

        # --- 検証 ---
        # 中央変位を取得 (target_nodeのY変位)
        if target_node != -1:
            idx = (target_node - 1) * 3 + 1
            disp_sim = U[idx]

            E_val = mat_props['E']; I_val = mat_props['I']
            delta_theory = (abs(P_load) * L_total**3) / (48 * E_val * I_val)

            print(f"\n--- 検証 ---")
            print(f"分割数: {num_div}")
            print(f"中央たわみ (解析値): {abs(disp_sim):.5f} mm")
            print(f"中央たわみ (理論値): {delta_theory:.5f} mm")

## **8.3 連続梁の解析（径間数を増やしてみる）**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 1. 可視化関数 (最大・最小値プロット機能付き：左右表示版)
# ==========================================
def plot_frame_forces(nodes, elements, U_all, element_forces, dist_loads={}, scale_factor=1.0):
    """
    解析結果（変形図、SFD、BMD）を描画する関数
    ※最大値・最小値をグラフの「左右」にプロットします
    """
    plt.figure(figsize=(10, 12))

    # --- 内部関数: 最大・最小値を左右に注釈する ---
    def add_peak_labels(ax, x_vals, y_vals, unit="", color='black'):
        x_arr = np.array(x_vals)
        y_arr = np.array(y_vals)

        # グラフの全幅を取得（左右判定用）
        x_min_global = np.min(x_arr)
        x_max_global = np.max(x_arr)
        x_center = (x_min_global + x_max_global) / 2.0

        # 注釈を描画するヘルパー関数
        def annotate_point(idx, label_prefix):
            x_target, y_target = x_arr[idx], y_arr[idx]

            # ピーク位置が「左寄り」なら「右」に文字を置く
            # ピーク位置が「右寄り」なら「左」に文字を置く
            if x_target < x_center:
                xytext_offset = (30, 0)   # 右へ30ポイント
                ha_align = 'left'         # 文字の左端を合わせる
                # 矢印を少しカーブさせる
                conn_style = 'arc3,rad=-0.2'
            else:
                xytext_offset = (-30, 0)  # 左へ30ポイント
                ha_align = 'right'        # 文字の右端を合わせる
                conn_style = 'arc3,rad=0.2'

            ax.annotate(f'{label_prefix}:\n{y_target:.2e} {unit}',
                        xy=(x_target, y_target),
                        xytext=xytext_offset,
                        textcoords='offset points',
                        ha=ha_align,
                        va='center', # 上下中央揃え
                        fontsize=9,
                        bbox=dict(boxstyle='round,pad=0.3', fc='white', ec=color, alpha=0.9),
                        arrowprops=dict(arrowstyle='->', connectionstyle=conn_style, color=color))

        # 最大値 (Max)
        idx_max = np.argmax(y_arr)
        annotate_point(idx_max, "Max")

        # 最小値 (Min)
        idx_min = np.argmin(y_arr)
        annotate_point(idx_min, "Min")

    # --- (1) 変形図 ---
    ax1 = plt.subplot(3, 1, 1)

    nodes_def = {}
    y_coords_def = []

    # データ収集用リスト
    all_def_x = []
    all_def_y = []

    for i, (x, y) in nodes.items():
        u = U_all[(i-1)*3]; v = U_all[(i-1)*3 + 1]
        dx_val = x + u * scale_factor
        dy_val = y + v * scale_factor
        nodes_def[i] = (dx_val, dy_val)
        y_coords_def.append(dy_val)

    for el_id, (i, j) in elements.items():
        x = [nodes[i][0], nodes[j][0]]; y = [nodes[i][1], nodes[j][1]]
        plt.plot(x, y, 'k--', lw=1, alpha=0.3)

        # 変形後座標
        x_def = [nodes_def[i][0], nodes_def[j][0]]; y_def = [nodes_def[i][1], nodes_def[j][1]]
        plt.plot(x_def, y_def, 'r-', lw=2)

        # プロット用データに追加
        all_def_x.extend(x_def)
        all_def_y.extend(y_def)

    plt.title(f'Deformation (Scale: {scale_factor}x / Real Value)')
    plt.grid(True); plt.ylabel('Y (mm)')

    # Y軸範囲調整
    y_min_val, y_max_val = min(y_coords_def), max(y_coords_def)
    y_range = y_max_val - y_min_val
    if y_range == 0: y_range = 1.0
    margin = y_range * 0.2
    plt.ylim(y_min_val - margin, y_max_val + margin)

    # ★最大・最小プロット
    add_peak_labels(ax1, all_def_x, all_def_y, "mm", color='red')

    # --- 内部関数: 分布計算 ---
    def get_element_distribution(el_id, i, j, forces, w):
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)
        Q_i = forces['Q_i']; M_i = forces['M_i']

        num_points = 21
        x_local = np.linspace(0, L, num_points)
        Q_vals = Q_i + w * x_local
        M_vals = -M_i + Q_i * x_local + 0.5 * w * x_local**2
        X_vals = np.linspace(xi, xj, num_points)
        return X_vals, Q_vals, M_vals

    # --- (2) SFD ---
    ax2 = plt.subplot(3, 1, 2)
    all_sfd_x = []
    all_sfd_y = []

    for el_id, (i, j) in elements.items():
        w = dist_loads.get(el_id, 0.0)
        X_vals, Q_vals, _ = get_element_distribution(el_id, i, j, element_forces[el_id], w)

        plt.plot(X_vals, [0]*len(X_vals), 'k-', lw=0.5, alpha=0.5)
        plt.plot(X_vals, Q_vals, 'b-', lw=2)
        plt.fill_between(X_vals, 0, Q_vals, alpha=0.2, color='blue')

        all_sfd_x.extend(X_vals)
        all_sfd_y.extend(Q_vals)

    plt.title('Shear Force Diagram (SFD) [N]')
    plt.grid(True); plt.ylabel('Shear (N)')

    # ★最大・最小プロット
    add_peak_labels(ax2, all_sfd_x, all_sfd_y, "N", color='blue')

    # --- (3) BMD ---
    ax3 = plt.subplot(3, 1, 3)
    all_bmd_x = []
    all_bmd_y = []

    for el_id, (i, j) in elements.items():
        w = dist_loads.get(el_id, 0.0)
        X_vals, _, M_vals = get_element_distribution(el_id, i, j, element_forces[el_id], w)

        # 引張側正（符号反転）
        plt.plot(X_vals, [0]*len(X_vals), 'k-', lw=0.5, alpha=0.5)
        plt.plot(X_vals, -M_vals, 'g-', lw=2)
        plt.fill_between(X_vals, 0, -M_vals, alpha=0.2, color='green')

        all_bmd_x.extend(X_vals)
        all_bmd_y.extend(-M_vals)

    plt.title('Bending Moment Diagram (BMD) [N.mm] (Tension Side)')
    plt.grid(True); plt.xlabel('X (mm)'); plt.ylabel('Moment')

    # ★最大・最小プロット
    add_peak_labels(ax3, all_bmd_x, all_bmd_y, "N.mm", color='green')

    plt.tight_layout()
    plt.show()

# ==========================================
# 2. フレーム解析ソルバー
# ==========================================
def solve_frame(nodes, elements, materials, bcs, loads, dist_loads={}):
    print("--- 解析開始 ---")
    num_nodes = len(nodes); num_dofs = num_nodes * 3
    K_all = np.zeros((num_dofs, num_dofs)); F_all = np.zeros(num_dofs)

    # 1. 剛性行列アセンブリ
    for el_id, (i, j) in elements.items():
        E = materials[el_id]['E']; A = materials[el_id]['A']; I = materials[el_id]['I']
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)
        c = dx / L; s = dy / L

        k_local = np.zeros((6, 6))
        k_local[0,0] = k_local[3,3] = E*A/L; k_local[0,3] = k_local[3,0] = -E*A/L
        k1 = 12*E*I/L**3; k2 = 6*E*I/L**2; k3 = 4*E*I/L; k4 = 2*E*I/L
        k_local[1,1]=k_local[4,4]=k1; k_local[1,4]=k_local[4,1]=-k1
        k_local[1,2]=k_local[2,1]=k2; k_local[1,5]=k_local[5,1]=k2
        k_local[4,2]=k_local[2,4]=-k2; k_local[4,5]=k_local[5,4]=-k2
        k_local[2,2]=k_local[5,5]=k3; k_local[2,5]=k_local[5,2]=k4

        T = np.zeros((6, 6)); rot = np.array([[c, s, 0], [-s, c, 0], [0, 0, 1]])
        T[0:3, 0:3] = rot; T[3:6, 3:6] = rot
        K_el = T.T @ k_local @ T

        indices = [ (i-1)*3, (i-1)*3+1, (i-1)*3+2, (j-1)*3, (j-1)*3+1, (j-1)*3+2 ]
        for r in range(6):
            for c_ in range(6):
                K_all[indices[r], indices[c_]] += K_el[r, c_]

    # 2. 荷重ベクトル作成 (分布荷重 -> 等価節点荷重)
    for el_id, w in dist_loads.items():
        if w == 0: continue
        i, j = elements[el_id]
        xi, yi = nodes[i]; xj, yj = nodes[j]
        L = np.sqrt((xj-xi)**2 + (yj-yi)**2)
        Fy_eq = w * L / 2.0
        Mi_eq = -w * L**2 / 12.0
        Mj_eq = +w * L**2 / 12.0
        idx_i = (i-1)*3; idx_j = (j-1)*3
        F_all[idx_i+1] += Fy_eq; F_all[idx_i+2] += Mi_eq
        F_all[idx_j+1] += Fy_eq; F_all[idx_j+2] += Mj_eq

    # 3. 求解
    fixed_dofs = []
    for node_id, constraint in bcs.items():
        idx = (node_id - 1) * 3
        if constraint[0]: fixed_dofs.append(idx)
        if constraint[1]: fixed_dofs.append(idx+1)
        if constraint[2]: fixed_dofs.append(idx+2)

    free_dofs = [i for i in range(num_dofs) if i not in fixed_dofs]
    K_ff = K_all[np.ix_(free_dofs, free_dofs)]
    F_f = F_all[free_dofs]

    try:
        U_f = np.linalg.solve(K_ff, F_f)
    except:
        print("Error: Singular Matrix"); return None, None

    U_all = np.zeros(num_dofs); U_all[free_dofs] = U_f

    # 4. 断面力計算
    element_forces = {}
    for el_id, (i, j) in elements.items():
        E = materials[el_id]['E']; A = materials[el_id]['A']; I = materials[el_id]['I']
        xi, yi = nodes[i]; xj, yj = nodes[j]
        dx = xj - xi; dy = yj - yi; L = np.sqrt(dx**2 + dy**2)
        c = dx / L; s = dy / L

        u_vec = np.array([U_all[(i-1)*3], U_all[(i-1)*3+1], U_all[(i-1)*3+2],
                          U_all[(j-1)*3], U_all[(j-1)*3+1], U_all[(j-1)*3+2]])
        T = np.zeros((6, 6)); rot = np.array([[c, s, 0], [-s, c, 0], [0, 0, 1]])
        T[0:3, 0:3] = rot; T[3:6, 3:6] = rot
        u_local = T @ u_vec

        k_local = np.zeros((6, 6))
        k_local[0,0] = k_local[3,3] = E*A/L; k_local[0,3] = k_local[3,0] = -E*A/L
        k1 = 12*E*I/L**3; k2 = 6*E*I/L**2; k3 = 4*E*I/L; k4 = 2*E*I/L
        k_local[1,1]=k_local[4,4]=k1; k_local[1,4]=k_local[4,1]=-k1
        k_local[1,2]=k_local[2,1]=k2; k_local[1,5]=k_local[5,1]=k2
        k_local[4,2]=k_local[2,4]=-k2; k_local[4,5]=k_local[5,4]=-k2
        k_local[2,2]=k_local[5,5]=k3; k_local[2,5]=k_local[5,2]=k4

        f_disp = k_local @ u_local

        w = dist_loads.get(el_id, 0.0)
        f_fixed = np.zeros(6)
        if w != 0:
            Ry = -w * L / 2.0
            Mi = +w * L**2 / 12.0
            Mj = -w * L**2 / 12.0
            f_fixed = np.array([0, Ry, Mi, 0, Ry, Mj])

        f_total = f_disp + f_fixed
        element_forces[el_id] = {
            'N_j': f_total[3], 'Q_i': f_total[1], 'M_i': f_total[2], 'Q_j': f_total[4], 'M_j': f_total[5]
        }
    return U_all, element_forces

# ==========================================
# 3. メイン実行ブロック (3径間連続梁)
# ==========================================
if __name__ == "__main__":
    # --- 問題設定 (単位: N, mm) ---
    # 3径間: 50m + 50m + 50m = 150m (150,000 mm)
    # 荷重: q = 79.4 kN/m = 79.4 N/mm

    L_span = 50000.0   # 1径間の長さ (mm)
    num_spans = 3      # 径間数
    q_load = -79.4     # 分布荷重 (N/mm, 下向きは負)

    div_per_span = 10  # 1径間あたりの要素分割数 (滑らかさのため)
    dx = L_span / div_per_span

    # --- 1. モデル生成 ---
    nodes = {}
    elements = {}
    materials = {}
    dist_loads = {}
    bcs = {}
    loads = {} # 集中荷重用（今回は空）

    # 巨大構造物用の仮定断面 (剛性は分布形状には影響しないが、変位量には影響する)
    # ここでは十分剛な値を仮定
    mat_props = {
        'E': 2.0e5,    # 鋼材想定のまま (N/mm^2)
        'A': 1.0e6,    # 断面積 (mm^2)
        'I': 4.0e12    # 1.0e11 * 40 = 4.0e12 (mm^4)
    }

    total_nodes = num_spans * div_per_span + 1

    for i in range(total_nodes):
        # 節点生成
        x_coord = i * dx
        nodes[i+1] = (x_coord, 0.0)

        # 境界条件設定 (支点位置判定)
        # x = 0, 50000, 100000, 150000 の位置に支点
        # 浮動小数点比較のため少し余裕を持たせるか、整数ロジックで判定
        if i % div_per_span == 0:
            # 支点位置
            if i == 0:
                bcs[i+1] = (True, True, False) # 左端: ピン (X, Y固定)
                print(f"Node {i+1} (x={x_coord/1000}m): Pin Support")
            else:
                bcs[i+1] = (False, True, False) # 中間・右端: ローラー (Y固定)
                print(f"Node {i+1} (x={x_coord/1000}m): Roller Support")
        else:
            bcs[i+1] = (False, False, False) # フリー

        # 要素・荷重生成
        if i < total_nodes - 1:
            el_id = i + 1
            elements[el_id] = (i+1, i+2)
            materials[el_id] = mat_props
            dist_loads[el_id] = q_load # 全要素に分布荷重

    # --- 2. 解析実行 ---
    U, F = solve_frame(nodes, elements, materials, bcs, loads, dist_loads)

    if U is not None:
        # 倍率は1.0 (実寸)
        scale_factor = 1.0
        print(f"Visualization Scale Factor: {scale_factor}x (Real Scale)")

        # 描画
        plot_frame_forces(nodes, elements, U, F, dist_loads, scale_factor=scale_factor)

        # --- 検証用出力 (値が巨大なためMN単位などで読み替えてください) ---
        print("\n--- 解析終了 ---")
        print("※単位注意: 力は[N]、モーメントは[N.mm]で出力されています。")
        print(f"例: 1,000,000 N = 1 MN")